# **1. Import Library**

In [31]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Pada tahap ini, Anda perlu mengimpor beberapa pustaka (library) Python yang dibutuhkan untuk analisis data dan pembangunan model machine learning.

In [46]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

# **2. Memuat Dataset dari Hasil Clustering**

Memuat dataset hasil clustering dari file CSV ke dalam variabel DataFrame.

In [33]:
df=pd.read_csv('/content/drive/MyDrive/SPK/DBS/hasil_clustering.csv')
df

,Hours_Studied,Attendance,Sleep_Hours,Previous_Scores,Tutoring_Sessions,Physical_Activity,Exam_Score,Cluster,Extracurricular_Activities,Internet_Access,...,Gender,School_Type,Parental_Involvement,Access_to_Resources,Motivation_Level,Family_Income,Teacher_Quality,Peer_Influence,Parental_Education_Level,Distance_from_Home
0,23,84,7,73,0,3,67,0,Yes,Yes,...,Male,Public,Low,High,Low,Low,Medium,Positive,High School,Near
1,19,64,8,59,2,4,61,2,Yes,Yes,...,Male,Public,Low,Medium,Low,Medium,Medium,Negative,College,Moderate
2,24,98,7,91,2,4,74,3,Yes,Yes,...,Male,Public,Medium,Medium,Medium,Medium,Medium,Neutral,Postgraduate,Near
3,29,89,8,98,1,4,71,3,Yes,Yes,...,Male,Public,Low,Medium,Medium,Medium,Medium,Negative,High School,Moderate
4,19,92,6,65,3,4,70,0,Yes,Yes,...,Male,Public,Medium,Medium,Medium,Medium,High,Neutral,College,Near
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6373,25,69,7,76,1,2,68,1,Yes,Yes,...,Male,Public,High,Medium,Medium,High,Medium,Positive,High School,Near
6374,23,76,8,81,3,2,69,1,Yes,Yes,...,Male,Public,High,Medium,Medium,Low,High,Positive,High School,Near
6375,20,90,6,65,3,2,68,0,Yes,Yes,...,Male,Public,Medium,Low,Low,Low,Medium,Negative,Postgraduate,Near
6376,10,86,6,91,2,3,68,3,Yes,Yes,...,Male,Public,High,High,High,Low,Medium,Positive,High School,Far


In [34]:
# 1. Binary Encoding (menggunakan One-Hot Encoding dengan drop_first=True)
binary_cols = ['Extracurricular_Activities', 'Internet_Access', 'Learning_Disabilities',
               'Gender', 'School_Type']
df = pd.get_dummies(df, columns=binary_cols, drop_first=True, dtype=int)

# 2. Label Encoding (lebih dari 2 kategori menjadi One-Hot Encoding)
multi_cat_cols = ['Parental_Involvement', 'Access_to_Resources', 'Motivation_Level',
                  'Family_Income', 'Teacher_Quality', 'Peer_Influence',
                  'Parental_Education_Level', 'Distance_from_Home']
df = pd.get_dummies(df, columns=multi_cat_cols, drop_first=False, dtype=int)

# Menampilkan hasil encoding
df

,Hours_Studied,Attendance,Sleep_Hours,Previous_Scores,Tutoring_Sessions,Physical_Activity,Exam_Score,Cluster,Parental_Involvement_High,Parental_Involvement_Low,...,Teacher_Quality_Medium,Peer_Influence_Negative,Peer_Influence_Neutral,Peer_Influence_Positive,Parental_Education_Level_College,Parental_Education_Level_High School,Parental_Education_Level_Postgraduate,Distance_from_Home_Far,Distance_from_Home_Moderate,Distance_from_Home_Near
0,23,84,7,73,0,3,67,0,0,1,...,1,0,0,1,0,1,0,0,0,1
1,19,64,8,59,2,4,61,2,0,1,...,1,1,0,0,1,0,0,0,1,0
2,24,98,7,91,2,4,74,3,0,0,...,1,0,1,0,0,0,1,0,0,1
3,29,89,8,98,1,4,71,3,0,1,...,1,1,0,0,0,1,0,0,1,0
4,19,92,6,65,3,4,70,0,0,0,...,0,0,1,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6373,25,69,7,76,1,2,68,1,1,0,...,1,0,0,1,0,1,0,0,0,1
6374,23,76,8,81,3,2,69,1,1,0,...,0,0,0,1,0,1,0,0,0,1
6375,20,90,6,65,3,2,68,0,0,0,...,1,1,0,0,0,0,1,0,0,1
6376,10,86,6,91,2,3,68,3,1,0,...,1,0,0,1,0,1,0,1,0,0


# **3. Data Splitting**

Tahap Data Splitting bertujuan untuk memisahkan dataset menjadi dua bagian: data latih (training set) dan data uji (test set).

In [44]:
# Memisahkan fitur (X) dan target label (y)
clustering = df.drop(columns=['Cluster'])  # Fitur (tanpa kolom target 'Cluster')
target = df['Cluster']             # Target label (Cluster)

# Membagi dataset menjadi data latih (70%) dan data uji (30%)
df_train, df_test, target_train, target_test = train_test_split(clustering, target, test_size=0.3, random_state=42)

# Menampilkan ukuran data latih dan data uji
print("Ukuran Data Latih:", df_train.shape)
print("Ukuran Data Uji:", df_test.shape)

Ukuran Data Latih: (4464, 31)
Ukuran Data Uji: (1914, 31)


# **4. Membangun Model Klasifikasi**


## **a. Membangun Model Klasifikasi**

Setelah memilih algoritma klasifikasi yang sesuai, langkah selanjutnya adalah melatih model menggunakan data latih.

Berikut adalah rekomendasi tahapannya.
1. Pilih algoritma klasifikasi yang sesuai, seperti Logistic Regression, Decision Tree, Random Forest, atau K-Nearest Neighbors (KNN).
2. Latih model menggunakan data latih.

In [63]:
# Model Klasifikasi 1: Random Forest
rf_classifier = RandomForestClassifier(random_state=42)
rf_classifier.fit(df_train, target_train)
rf_predictions_before = rf_classifier.predict(df_test)

Random Forest efektif dalam menangani data dengan banyak fitur, mengurangi risiko overfitting, dan memberikan estimasi pentingnya fitur.

In [64]:
# Model Klasifikasi 2: K-Nearest Neighbors (KNN) dengan k=4
knn_classifier = KNeighborsClassifier(n_neighbors=4)
knn_classifier.fit(df_train, target_train)
knn_predictions_before = knn_classifier.predict(df_test)

KNN menentukan kelas suatu data berdasarkan mayoritas kelas dari k tetangga terdekatnya. Dalam implementasi ini, nilai k ditentukan sebanyak 4. Jarak antara data dihitung menggunakan jarak Euclidean secara default.

## **b. Evaluasi Model Klasifikasi**

Berikut adalah **rekomendasi** tahapannya.
1. Lakukan prediksi menggunakan data uji.
2. Hitung metrik evaluasi seperti Accuracy dan F1-Score (Opsional: Precision dan Recall).
3. Buat confusion matrix untuk melihat detail prediksi benar dan salah.

In [66]:
# Menghitung metrik evaluasi untuk Random Forest sebelum tuning
rf_metrics_before = {
    'Model': 'Random Forest (Before Tuning)',
    'Accuracy': accuracy_score(target_test, rf_predictions_before),
    'Precision': precision_score(target_test, rf_predictions_before, average='weighted'),
    'Recall': recall_score(target_test, rf_predictions_before, average='weighted'),
    'F1-Score': f1_score(target_test, rf_predictions_before, average='weighted')
}

# Menghitung metrik evaluasi untuk KNN sebelum tuning
knn_metrics_before = {
    'Model': 'KNN (Before Tuning)',
    'Accuracy': accuracy_score(target_test, knn_predictions_before),
    'Precision': precision_score(target_test, knn_predictions_before, average='weighted'),
    'Recall': recall_score(target_test, knn_predictions_before, average='weighted'),
    'F1-Score': f1_score(target_test, knn_predictions_before, average='weighted')
}

# Menyusun hasil evaluasi dalam bentuk DataFrame untuk perbandingan
comparison_before_df = pd.DataFrame([rf_metrics_before, knn_metrics_before])

comparison_before_df

,Model,Accuracy,Precision,Recall,F1-Score
0,Random Forest (Before Tuning),0.994253,0.994255,0.994253,0.994250
1,KNN (Before Tuning),0.969697,0.969978,0.969697,0.969684


##Analisis Perbandingan:
- Akurasi: Random Forest memiliki akurasi lebih tinggi (99.4%) dibandingkan dengan KNN (96.9%). Hal ini menunjukkan bahwa Random Forest lebih efektif dalam mengklasifikasikan data uji secara keseluruhan.

- Precision: Precision yang lebih tinggi pada Random Forest menunjukkan bahwa model ini lebih sedikit menghasilkan false positives dibandingkan dengan KNN.

- Recall: Random Forest juga memiliki recall yang lebih tinggi, menunjukkan bahwa model ini lebih mampu menangkap kasus positif dengan benar.

- F1-Score: Karena nilai F1-Score dari Random Forest lebih tinggi, model ini menunjukkan keseimbangan yang lebih baik antara precision dan recall.

**Kesimpulan**: Berdasarkan hasil evaluasi ini, Random Forest menunjukkan performa yang lebih baik dalam mengklasifikasikan data dibandingkan KNN.

## **c. Tuning Model Klasifikasi (Optional)**

Gunakan GridSearchCV, RandomizedSearchCV, atau metode lainnya untuk mencari kombinasi hyperparameter terbaik

In [48]:
# Hyperparameter Tuning untuk Random Forest dengan GridSearchCV
param_grid_rf = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

grid_search_rf = GridSearchCV(estimator=RandomForestClassifier(random_state=42),
                              param_grid=param_grid_rf,
                              cv=3,
                              n_jobs=-1,
                              verbose=2)

grid_search_rf.fit(df_train, target_train)

print("Best Hyperparameters for Random Forest:", grid_search_rf.best_params_)
print("Best Score (Random Forest):", grid_search_rf.best_score_)

Fitting 3 folds for each of 108 candidates, totalling 324 fits
Best Hyperparameters for Random Forest: {'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Best Score (Random Forest): 0.996415770609319


In [50]:
#  Hyperparameter Tuning untuk KNN dengan RandomizedSearchCV
param_dist_knn = {
    'n_neighbors': [3, 5, 7, 9],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'minkowski']
}

random_search_knn = RandomizedSearchCV(estimator=KNeighborsClassifier(),
                                       param_distributions=param_dist_knn,
                                       n_iter=10,
                                       cv=3,
                                       random_state=42,
                                       n_jobs=-1,
                                       verbose=2)

random_search_knn.fit(df_train, target_train)

print("Best Hyperparameters for KNN:", random_search_knn.best_params_)
print("Best Score (KNN):", random_search_knn.best_score_)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
Best Hyperparameters for KNN: {'weights': 'distance', 'n_neighbors': 7, 'metric': 'manhattan'}
Best Score (KNN): 0.9670698924731184


## **d. Evaluasi Model Klasifikasi setelah Tuning (Optional)**

Berikut adalah rekomendasi tahapannya.
1. Gunakan model dengan hyperparameter terbaik.
2. Hitung ulang metrik evaluasi untuk melihat apakah ada peningkatan performa.

In [68]:
# Model Random Forest setelah tuning
rf_best = RandomForestClassifier(
    max_depth=20,
    min_samples_leaf=1,
    min_samples_split=2,
    n_estimators=200,
    random_state=42
)
rf_best.fit(df_train, target_train)
rf_predictions_after = rf_best.predict(df_test)

In [69]:
# Model KNN setelah tuning
knn_best = KNeighborsClassifier(
    weights='distance',
    n_neighbors=7,
    metric='manhattan'
)
knn_best.fit(df_train, target_train)
knn_predictions_after = knn_best.predict(df_test)

In [71]:
# Menghitung metrik evaluasi untuk Random Forest setelah tuning
rf_metrics_after = {
    'Model': 'Random Forest (After Tuning)',
    'Accuracy': accuracy_score(target_test, rf_predictions_after),
    'Precision': precision_score(target_test, rf_predictions_after, average='weighted'),
    'Recall': recall_score(target_test, rf_predictions_after, average='weighted'),
    'F1-Score': f1_score(target_test, rf_predictions_after, average='weighted')
}

# Menghitung metrik evaluasi untuk KNN setelah tuning
knn_metrics_after = {
    'Model': 'KNN (After Tuning)',
    'Accuracy': accuracy_score(target_test, knn_predictions_after),
    'Precision': precision_score(target_test, knn_predictions_after, average='weighted'),
    'Recall': recall_score(target_test, knn_predictions_after, average='weighted'),
    'F1-Score': f1_score(target_test, knn_predictions_after, average='weighted')
}
# Menyusun hasil evaluasi dalam bentuk DataFrame untuk perbandingan
results = pd.DataFrame([rf_metrics_before, knn_metrics_before, rf_metrics_after, knn_metrics_after])

# Menampilkan tabel perbandingan
results

,Model,Accuracy,Precision,Recall,F1-Score
0,Random Forest (Before Tuning),0.994253,0.994255,0.994253,0.994250
1,KNN (Before Tuning),0.969697,0.969978,0.969697,0.969684
2,Random Forest (After Tuning),0.994775,0.994775,0.994775,0.994773
3,KNN (After Tuning),0.971264,0.971442,0.971264,0.971261


## **e. Analisis Hasil Evaluasi Model Klasifikasi**

Analisis dan Pembandingan:
1. Peningkatan Performa Setelah Tuning:

- Random Forest: Setelah tuning, performa Random Forest mengalami sedikit peningkatan pada semua metrik evaluasi, dengan Accuracy meningkat dari 0.994253 menjadi 0.994775, dan F1-Score meningkat dari 0.994250 menjadi 0.994773.

- KNN: Kinerja KNN juga meningkat setelah tuning, dengan Accuracy meningkat dari 0.969697 menjadi 0.971264, dan F1-Score meningkat dari 0.969684 menjadi 0.971261.

2. Identifikasi Kelemahan Model:

Kelemahan pada KNN: Meskipun ada peningkatan, KNN tetap memiliki performa lebih rendah dibandingkan Random Forest, dengan Accuracy dan F1-Score yang lebih rendah. Ini mungkin menunjukkan bahwa KNN kurang efektif untuk dataset ini

Precision dan Recall: Kedua model menunjukkan keseimbangan yang baik antara Precision dan Recall. Namun, untuk model KNN lebih kecil dari Random Forest.

3. Overfitting atau Underfitting:

- Random Forest: Random Forest tampaknya tidak mengalami overfitting, karena perbedaan antara performa sebelum dan setelah tuning tidak terlalu besar. Model ini sudah sangat baik sebelum tuning dan hanya sedikit ditingkatkan.

- KNN: KNN mungkin menunjukkan sedikit overfitting, meskipun ada peningkatan pada metrik, kinerja model KNN yang lebih rendah dibandingkan Random Forest mungkin menunjukkan bahwa model ini tidak cukup robust untuk menangani data dalam jumlah besar atau data dengan kompleksitas tinggi.

4. Rekomendasi Tindakan Lanjutan:
- Mengumpulkan Data Tambahan:Jika dataset saat ini tidak cukup besar atau bervariasi, dapat untuk mengumpulkan lebih banyak data. Data tambahan dapat membantu model belajar lebih baik tentang variasi dalam data dan meningkatkan akurasi.

- Eksperimen dengan Algoritma Lain:
Meskipun Random Forest memberikan hasil terbaik, cobalah algoritma lain seperti Gradient Boosting atau Support Vector Machines (SVM) untuk membandingkan performa.

- Tuning Hyperparameter Lebih Lanjut:
Walaupun sudah ada peningkatan setelah tuning, dapat dilakukan dengan algoritma yang lain untuk mencari hyperparameter yang mungkin lebih optimal.